# Text2Cypher Retriever

Vector retrievers are great for finding relevant data based on semantic similarity or keyword matching.

To answer more specific questions, you may need to perform more complex queries to find data relating to specific nodes, relationships, or properties.

For example, you want to find:
- What asset manager owns a specific organization.
- How many stock types there are.
- What organizations are exposed to a certain risk factor. 

Text to Cypher retrievers allow you to convert natural language queries into Cypher queries that can be executed against the graph.

---

You will use the `Text2CypherRetriever` class to create a new retriever and use it in a `GraphRAG` pipeline.

Import the required Python modules, load the environment variables, create the connection to the graph, the LLM, and the embedding model.

In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.schema import get_schema

# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USER = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# --- Initialize LLM and Embedder ---
llm = OpenAILLM(model_name='gpt-4o', api_key=OPENAI_API_KEY)
embedder = OpenAIEmbeddings(api_key=OPENAI_API_KEY)


The `Text2CypherRetriever` automatically generates Cypher queries from natural language questions.

**How it works:**
- The retriever uses a Large Language Model (LLM) to translate your plain-English query into a Cypher query, based on your Neo4j schema.
- The schema is provided as a string describing the main node types and relationships in your graph (e.g., companies, risk factors, asset managers).

You can view the schema using the `get_schema` method.

In [ ]:
schema = get_schema(driver)
print(schema)

Create the `Text2CypherRetriever` using the Neo4j `driver`, `llm`, and the `schema`.

In [ ]:
# --- Text2CypherRetriever Example ---
text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    neo4j_database=NEO4J_DATABASE,
    llm=llm,
    neo4j_schema=schema
)

You run the retriever by passing a natural language query, for example "What companies are owned by BlackRock Inc.".

The retriever then:

1. Generates a corresponding Cypher query using the `schema` and the `llm`.
2. Executes the Cypher query using the `driver`.
3. Returns the generated Cypher and the results.

In [ ]:
query = "What companies are owned by BlackRock Inc."
cypher_query = text2cypher_retriever.get_search_results(query)

print("Original Query:", query)
print("Generated Cypher:", cypher_query.metadata["cypher"])

print("Cypher Query Results:")
for record in cypher_query.records:
    print(record)

> **Tip:**
> You can configure the retriever to only look at part of the graph by removing node labels and relationship types from the `schema`.

You can use the `Text2CypherRetriever` retriever as part of a `GraphRAG` pipeline. The `GraphRAG` pipeline will generate responses based on the original `query` and the results return by the generated Cypher query. 

In [ ]:
# --- Initialize RAG and Perform Search ---
query = "Who are the assets managers?"
rag = GraphRAG(llm=llm, retriever=text2cypher_retriever)
response = rag.search(
    query,
    return_context=True
    )
print(response.answer)

In [ ]:
# View the generated Cypher and results used in this query
print("Generate Cypher:", response.retriever_result.metadata["cypher"])
print("Context:", *response.retriever_result.items, sep="\n")

Text2Cypher retrievers allow you to answer more specific questions and gain fact based context from the graph, they also:

- Remove the need to manually write Cypher for each question.
- Make graph querying accessible to non-technical users.
- Support rapid prototyping, exploration, and building natural language interfaces to your knowledge graph.

> **Tip:**
> You can tailor the generated Cypher by providing `examples` of user and Cypher queries when creating the `Text2CypherRetriever`.

---

Experiment with the Text2Cypher retriever and GraphRAG pipeline, review the answers and the generated Cypher. 

[View the complete code](solutions/01_03_text2cypher_retriever.py)